## Group Assignment
### Team Number: 20
### Team Member Names: Ricky Qin, Tong Liu, Samay Bhagat, Alexander Domilescu
### Team Strategy Chosen: RISKY

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Ricky Qin, Tong Liu, Samay Bhagat, Alexander Domilescu

In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

The following code creates a function "currency_exchange" that takes in a dataframe and the currency of the dataframe and convert it into CAD.

In [2]:
team_number = 20
startdate = '2023-01-01'
enddate = '2023-10-31'

ticker_file = 'Tickers_Example.csv' 
tickers = pd.read_csv(ticker_file, header=None).values.flatten().tolist() 

#currency conversion
CAD_transfer = yf.Ticker('CADUSD=x')
transferRate = CAD_transfer.history(start=startdate, end=enddate)
CAD_to_USD = transferRate[['Close']]
CAD_to_USD.index = CAD_to_USD.index.strftime('%Y-%m-%d')
    
def currency_exchange(base, currency):
    base.index = base.index.strftime('%Y-%m-%d')
    if currency == 'USD':
        exchange_rates = pd.DataFrame()
        exchange_rates['Close'] = base['Close']/CAD_to_USD['Close']
        exchange_rates = exchange_rates.dropna()
        return exchange_rates
    else:
        return base

In [3]:
# checks if the stock is denominated in either CAD or USD
def is_valid_currency(ticker):
    stock_info = yf.Ticker(ticker).info
    currency = stock_info.get('currency', '').upper() # retrieving the stock currency
    return currency == 'USD' or currency == 'CAD' # returns T/F based on currency

In [4]:
#gets stock data (info, close prices, volume) for a ticker
def get_stock_data(ticker):
    # get stock info
    stock_info = yf.Ticker(ticker).info
    
    # check if the currency is valid
    if is_valid_currency(ticker):
        # download historical data
        data = yf.download(ticker, start=startdate, end=enddate)
        
        # returning daily volume
        monthly_volume = data['Volume'].resample('D').sum()
        
        #turning American stocks to CAD$
        stock_price = currency_exchange(data[['Close']], stock_info['currency'])
        
        # create a dictionary for the ticker
        result = {
            'ticker': ticker,
            'info': stock_info,
            'close': stock_price,
            'volume': pd.DataFrame(monthly_volume)
        }
        return result
    else:
        print(f"Skipping {ticker}: Invalid currency.")
        return None

# accumulates a list of dictionaries through get_stock_data    
def process_tickers(ticker_list):
    result_list = []
    for ticker in ticker_list:
        stock_data = get_stock_data(ticker)
        if stock_data is not None:
            result_list.append(stock_data)
    return result_list

result_list = process_tickers(tickers)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Skipping AGN: Invalid currency.
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*

Filter_stocks will filter the data downloaded above based on the following criterion: denominated in USD or CAD, and an average monthly volume of at least 150,000 shares during the time interval of January 01, 2023 to October 31, 2023.

In [5]:
def filter_stocks(result_list, min_avg_volume=150000, min_trading_days=18):
    filtered_list = []

    for stock_data in result_list:
        ticker = stock_data['ticker']
        volume_data = stock_data['volume']

        # calculate average monthly volume
        monthly_avg_volume = {}
        for date, volume in volume_data.iterrows():
            month = date.strftime('%Y-%m')
            if month not in monthly_avg_volume:
                monthly_avg_volume[month] = {'total_volume': 0, 'days_counted': 0}
            if volume['Volume'] > 0 or monthly_avg_volume[month]['days_counted'] < min_trading_days:
                monthly_avg_volume[month]['total_volume'] += volume['Volume']
                monthly_avg_volume[month]['days_counted'] += 1

        # remove stocks with less than min_trading_days in any month or average volume less than min_avg_volume
        valid_months = [month for month, data in monthly_avg_volume.items() if data['days_counted'] >= min_trading_days]
        avg_monthly_volume = sum(monthly_avg_volume[month]['total_volume'] / data['days_counted'] for month, data in monthly_avg_volume.items() if data['days_counted'] > 0) / len(valid_months) if valid_months else 0

        if avg_monthly_volume >= min_avg_volume:
            filtered_list.append(stock_data)

    return filtered_list

filtered_list = filter_stocks(result_list)

The following code groups the filtered list of stocks into their respective sectors. It will then create a dictionary with the number of stocks in each sector and will sort the dictionary in descending order. This will be used to determine the correlation of the portfolio later on in the program.

In [6]:
#returns a list of industries 
def return_industry(tickers):
    industries = []
    for i in range (len (tickers)):
        stock = tickers[i]
        sector = stock["info"]['sector']
        industries.append(sector)
    return industries

list_of_industries = return_industry(filtered_list)

#returns a dictionary of sectors and how many companies are in the sector from a list of industries
def industry_count(industry_list):
    industry_dict={}
    while len(industry_list) > 0:
        industry_dict[industry_list[0]] = industry_list.count(industry_list[0])
        industry_list = remove_items(industry_list, industry_list[0])
    return industry_dict

#Used to remove existing industries
def remove_items(lst, item): 
    resulting_list = [i for i in lst if i != item] 
    return resulting_list

count_of_industries = industry_count(list_of_industries)

#Sorts the dictionary of industries by descending number of stocks in the industry
sorted_dict_of_industries = sorted(count_of_industries.items(),key=lambda x:x[1], reverse = True)

stock_grouper will take the stocks in the filtered list and put them in a dictionary seperated by their respective industries. We will then determine the volatility of each sector using a myriad of statistics.

Our strategy shown in the following code below is based around determining a aggregate score of the volatility of each industry. We will use the beta of the industry, the average standard deviation of the industry, the ratio between the 52 week high and low, and the overall correlation of the generated portfolio for this score. The higher the score of an industry, the more volatile it becomes, thus we will then add it to our final portfolio.

In [7]:
#creating a dictionary of stocks within the given industry
def stocks_in_industry (industry_dict):
    industry_portfolio = {}
    for i in range (len (industry_dict)):
        industry_portfolio[(industry_dict[i][0])] = stock_grouper(filtered_list, (industry_dict[i][0]))
    return industry_portfolio

#finds all stocks in the given industry and returns it in a list
def stock_grouper (stock_list, industry):
    stocks = []
    for i in range (len (stock_list)):
        stock = stock_list[i]
        sector = stock["info"]['sector']
        if sector == industry:
            stocks.append(stock)
    return stocks
            
stocks_in_industry_list = stocks_in_industry(sorted_dict_of_industries)

In [8]:
# This function calculates the beta of a ticker, relative to a market index
# For our purposes, we will use YTD daily data, since we want to assess how stocks perform relative to the market in
# the short term (we are only investing for 5 days)
# market data and stock data should have the same period and interval for consistency
def calculate_beta(market_data, stock_dict):
    # calculate market index returns
    returns = market_data
    returns = returns.rename(columns={'Close': 'Market'})
    returns = returns.pct_change()
    returns.drop(index=returns.index[0], inplace=True)
    
    # calculate returns for the stock
    stockReturns = stock_dict['close']
    stockReturns = stockReturns.rename(columns={'Close': stock_dict['ticker']})
    stockReturns = stockReturns.pct_change()
    stockReturns.drop(index=stockReturns.index[0], inplace=True)
    returns = returns.merge(stockReturns,left_index=True,right_index=True)
    
    # calculates covariance and market variance
    returnsCov = returns.cov()
    var = returns['Market'].var()
    beta = returnsCov/var
    
    return beta.iat[1,0]
    

In [9]:
# aggregate score for each industry is initially set to zero
# for each 'factor' we consider, we calculate the score for each industry and add it to its aggregate score
industry_scores = {}
for sector in stocks_in_industry_list:
    industry_scores[sector] = 0

The code below calculates the betas for each stock in each industry. The market that we chose to measure the stocks to is the S&P 500, simply because this index is one of the most comprehensive stock indexes available. Even though some of the stocks are Canadian, they tend to be related to the US market so it is still reasonable to compare them to the S&P 500.

In [10]:
# get market data (we are using s&p 500)
market_data = yf.Ticker('^GSPC').history(start=startdate, end=enddate)[['Close']]
market_data.index = market_data.index.strftime('%Y-%m-%d')

# consider beta for each industry (relative to s&p 500)
for sector in stocks_in_industry_list:
    stock_lst = stocks_in_industry_list[sector]
    beta_avg = 0
    for stock in stock_lst:
        ourbeta = calculate_beta(market_data,stock)
        beta_avg += ourbeta
    beta_avg /= len(stock_lst)
    industry_scores[sector] += beta_avg*0.3

In the code, we calculate the average standard deviations of daily returns for the stocks in each industry. A stock with a higher standard deviation tends to have larger daily fluctuations, thus making it more risky. These are the stocks that we want our portfolio to be comprised of.

In [11]:
for sector in stocks_in_industry_list:
    stock_lst = stocks_in_industry_list[sector]
    std_avg = 0
    for stock in stock_lst:
        close_pct = stock['close'].pct_change()
        close_pct.drop(index=close_pct.index[0], inplace=True)
        std_avg += close_pct['Close'].std()
    std_avg /= len(stock_lst)
    industry_scores[sector] += std_avg*20

The following code will compare the average 52 week high and low of each industry against each other. Our reasoning behind this statistic was to look at the previous volatility of the stock. If the resulting number was high, it would indicate a large potiential for movement and would signal a risky investment. If the resulting number was low, it would indicate a low potiential for movement and would signal a relatively stable stock that we do not want for our final portfolio.

In [12]:
for sector in stocks_in_industry_list:
    stock_lst = stocks_in_industry_list[sector]
    hilo_avg = 0
    for stock in stock_lst:
        hilo_avg += stock['info']['fiftyTwoWeekHigh']/stock['info']['fiftyTwoWeekLow']
    hilo_avg /= len(stock_lst)
    industry_scores[sector] += hilo_avg*0.1

In [13]:
# Adjust industry score by the number of stocks within
for sector in stocks_in_industry_list:
    stock_lst = stocks_in_industry_list[sector]
    industry_scores[sector] *= 1-2**(-len(stock_lst))

In [14]:
# convert industry_scores to a list so we can sort it
industry_scores_temp = []
for sector in industry_scores:
    industry_scores_temp.append((sector,industry_scores[sector]))
industry_scores = industry_scores_temp

# comparison function that sorts tuples by their second value in descending order
from functools import cmp_to_key
def sort_by_second_desc(a, b):
    if a[1] < b[1]:
        return 1
    elif a[1] > b[1]:
        return -1
    else:
        return 0
    
#sort industries by aggregate score in descending order
industry_scores.sort(key=cmp_to_key(sort_by_second_desc))

Now that we calculated the aggregate scores, we can rank the industries by how risky we determine them to be. We only want the riskiest ones so we consider those ones first until we reach the minimum requirement for number of stocks.

In [15]:
N_stocks = 10
principal = 750000
fee_per_stock = 4.95 # flat fee for each stock

#stock_weightings[i] is the weight of the ith stock considered
stock_weightings = [0.2,0.2,0.2,0.1,0.05,0.05,0.05,0.05,0.05,0.05]
# a list of tuples, each tuple stores a ticker string and its corresponding portfolio weighting
stock_percentages = []
for industry in industry_scores:
    
    if len(stock_percentages) >= N_stocks:
        break

    industry_name = industry[0]
    stock_lst = stocks_in_industry_list[industry_name]
    
    # as a tie breaker, we first consider the stocks within the current industry with the highest std
    # we want the industry to be represented by the riskiest stocks
    stock_stds = []
    for stock in stock_lst:
        stock_returns = stock['close'].pct_change()
        stock_returns.drop(index=stock_returns.index[0], inplace=True)
        stock_stds.append((stock['ticker'],stock_returns['Close'].std()))
    stock_stds.sort(key=cmp_to_key(sort_by_second_desc))
    
    for stock in stock_stds:
        # once we reach the minimum stock number requirement, we stop
        if len(stock_percentages) >= N_stocks:
            break
        stock_percentages.append((stock[0],stock_weightings[len(stock_percentages)]))

# Format and print final result
principal -= N_stocks*fee_per_stock
Portfolio_Final = {'Ticker': [],
                   'Price': [],
                   'Currency': [],
                   'Shares': [],
                   'Value': [],
                   'Weight': []
                  }

curr_cad_usd = yf.Ticker('CADUSD=x').info['previousClose']
for stock in stock_percentages:
    stock_info = yf.Ticker(stock[0]).info
    curr_price = stock_info['currentPrice']
    Portfolio_Final['Ticker'].append(stock[0])
    Portfolio_Final['Price'].append(curr_price)
    Portfolio_Final['Currency'].append(stock_info['currency'])
    num_shares = 0
    if stock_info['currency'] == 'USD':
        num_shares = principal*stock[1]/(curr_price/curr_cad_usd)
    else:
        num_shares = principal*stock[1]/curr_price
    Portfolio_Final['Shares'].append(num_shares)
    Portfolio_Final['Value'].append(principal*stock[1])
    Portfolio_Final['Weight'].append(stock[1])
    
Portfolio_Final = pd.DataFrame(data=Portfolio_Final,index=[(i+1) for i in range(N_stocks)])
Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight
1,SHOP.TO,96.26,CAD,1558.176813,149990.100,0.20
2,QCOM,127.75,USD,857.419904,149990.100,0.20
3,TXN,153.59,USD,713.167476,149990.100,0.20
4,ACN,334.04,USD,163.955503,74995.050,0.10
5,AAPL,189.97,USD,144.148277,37497.525,0.05
6,USB,37.20,USD,736.124951,37497.525,0.05
7,PYPL,55.76,USD,491.102012,37497.525,0.05
8,AIG,65.21,USD,419.933264,37497.525,0.05
9,AXP,164.42,USD,166.548158,37497.525,0.05
10,BAC,29.73,USD,921.084701,37497.525,0.05


In [16]:
# output result to csv file
Stocks_Final = Portfolio_Final[['Ticker','Shares']]
Stocks_Final.to_csv('Stocks_Group_'+str(team_number)+'.csv')